<a href="https://colab.research.google.com/github/Data-to-Insight-Center/patra-toolkit/blob/feature%2Fhuggingface-integration/huggingface-integration/examples/notebooks/GettingStarted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Patra Toolkit

This notebook serves as a quickstart guide to help you learn how to:

- Load and preprocess the UCI Adult Dataset  
- Build and train a neural network in TensorFlow  
- Generate a comprehensive Model Card using the **Patra Toolkit**  

By the end of this tutorial, you’ll have a validated Model Card (in JSON format) that captures key information about your model, including fairness and explainability metrics.  

---

## 1. Environment Setup

### 1.1 Install Required Packages


In [1]:
!pip install tensorflow scikit-learn pandas patra_toolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 4.0 MB/s eta 0:00:00


### 1.2 Import Dependencies

In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from patra_toolkit import ModelCard, AIModel

---
## 2. Load and Inspect the Data

We’ll use the **UCI Adult Dataset**, a commonly used dataset to predict whether a person's income exceeds a certain threshold based on demographic factors.

In [3]:
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data = pd.read_csv(data_url, names=columns, header=None)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


---
## 3. Preprocessing

### 3.1 Encode Target Variable
We’ll encode the **income** column using `LabelEncoder`, transforming the categorical values (e.g., `>50K` and `<=50K`) into numerical labels.

In [4]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

### 3.2 One-Hot Encode Categorical Features
We’ll convert other categorical variables into **one-hot encoding**. We use the parameter `drop_first=True` to avoid dummy variable traps.

In [5]:
data = pd.get_dummies(data, drop_first=True, dtype=float)
data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.3 Train-Test Split
Next, we separate features (**X**) from the target (**y**) and then split into training and testing sets.

In [6]:
X = data.drop('income', axis=1).values
y = data['income'].values

print("List of columns after one-hot encoding:")
print(data.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

List of columns after one-hot encoding:
['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'marital-status_ Married-AF-spouse', 'marital-status_ Married-civ-spouse', 'marital-status_ Married-spouse-absent', 'marital-status_ Never-married', 'marital-status_ Separated', 'marital-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial'

---
## 4. Model Training

We define a simple feed-forward neural network in TensorFlow, compile it with an **Adam** optimizer, and fit it on our training set.

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


Epoch 1/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6669 - loss: 499.7343 - val_accuracy: 0.2365 - val_loss: 233.9744
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6758 - loss: 71.4167 - val_accuracy: 0.2365 - val_loss: 22.0188
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6807 - loss: 37.2719 - val_accuracy: 0.8035 - val_loss: 20.2309
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6804 - loss: 19.5329 - val_accuracy: 0.8027 - val_loss: 5.9562
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6836 - loss: 7.7849 - val_accuracy: 0.7904 - val_loss: 5.6894
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6962 - loss: 2.0517 - val_accuracy: 0.3029 - val_loss: 0.8733
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7273 - loss: 0.8859 - val_accuracy: 0.8058 - val_loss: 1.0513
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7497 - loss: 0.7084 -

---
## 5. Model Card Generation with Patra Toolkit

Now that we have a trained model, let’s create a **Model Card** to capture essential metadata.

1. **ModelCard** object contains high-level information about the model (description, use-cases, etc.).  
2. **AIModel** object contains details about the model architecture, performance metrics, ownership, and location.  

Afterward, we’ll demonstrate how to automatically populate the following fields:  
- **Requirements** (packages and versions)  
- **Fairness/Bias Analysis**  
- **Explainability/XAI Analysis**  

---
### 5.1 Create a Model Card


In [8]:
mc = ModelCard(
    name="UCI Adult Data Analysis model using Tensorflow",
    version="0.1",
    short_description="UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    full_description=(
        "We have trained a ML model using the tensorflow framework to predict income "
        "for the UCI Adult Dataset. We leverage this data to run the Patra model cards "
        "to capture metadata about the model as well as fairness and explainability metrics."
    ),
    keywords="uci adult, tensorflow, explainability, fairness, patra",
    author="Your Name",
    input_type="Tabular",
    category="classification",
    foundational_model="None"
)

# Input and output references
mc.input_data = "https://archive.ics.uci.edu/ml/datasets/adult"
mc.output_data = "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01"  # Update with your model path


### 5.2 Create an AIModel Instance

This object describes the **model** itself, capturing details like the model’s location, license, framework, and metrics.


In [9]:
ai_model = AIModel(
    name="Income prediction tensorflow model",
    version="0.1",
    description="Census classification problem using TensorFlow Neural Network using the UCI Adult Dataset",
    owner="Your Name or Organization",
    location="https://example.com/path-to-model",  # Update with the actual location if hosted
    license="BSD-3 Clause",
    framework="tensorflow",
    model_type="dnn",
    test_accuracy=accuracy
)

# Populate the model's architecture details
ai_model.populate_model_structure(model)

# Add extra metrics
ai_model.add_metric("Test loss", loss)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.001)
ai_model.add_metric("Input Shape", str(X_train.shape))

# Attach the AIModel object to the ModelCard
mc.ai_model = ai_model


### 5.3 Automatically Capture Requirements

`populate_requirements()` will parse your environment to identify installed packages and capture them under **environment/requirements** in the Model Card.


In [10]:
mc.populate_requirements()

### 5.4 Bias (Fairness) Analysis

Below, we show how to call the `populate_bias()` method, which takes the test dataset, predicted labels, and the feature on which you want to measure bias. For demonstration, we assume the “gender” feature is at index 58 in **X_test** (as determined after one-hot encoding).

- `feature_name`: "gender"  
- `protected_feature_data`: The specific column from your **X_test** that corresponds to "gender"  
- `model`: The trained TensorFlow model (not strictly needed to compute bias, but used in some advanced checks)


In [11]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()

mc.populate_bias(
    X_test,
    y_test,
    y_pred,
    "gender",           # Name you want displayed in the report
    X_test[:, 58],      # The slice of data that corresponds to gender
    model
)

print("Bias Analysis:\n", mc.bias_analysis)


204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Bias Analysis:
 {'demographic_parity_diff': 0.032400848225782965, 'equal_odds_difference': 0.03417438108252019}


### 5.5 Explainability (XAI) Analysis

Similarly, we can generate some basic SHAP-based interpretability metrics or feature attribution for a sample of inputs.
- `num_samples_to_explain`: 10 in our case  
- We provide `X_test[:10]` along with the actual column names from the dataset (minus the target column).

In [12]:
# Rebuild the list of columns used in training
x_columns = data.columns.tolist()
x_columns.remove('income')  # Remove the target

mc.populate_xai(
    X_test[:10],
    x_columns,
    model
)

print("Explainability Analysis:\n", mc.xai_analysis)


Explainability Analysis:
 {'capital_gain': 0.14158532669974697, 'fnlwgt': 0.015681726452377108, 'age': 0.00021690960559580313, 'hours_per_week': 0.00013091093136204657, 'relationship__Not_in_family': 7.917947239345974e-05, 'marital_status__Married_civ_spouse': 7.522924906677672e-05, 'education__HS_grad': 5.9253134661254366e-05, 'occupation__Exec_managerial': 5.8465765582191924e-05, 'marital_status__Never_married': 2.9966458678243015e-05, 'education_num': 2.848242719968154e-05}


---
## 6. Validate and Save the Model Card

Before saving, let’s ensure our card follows Patra’s default schema by calling `mc.validate()`. If all checks pass, you can save it locally as a JSON file and later upload it to the **Patra Knowledge Base**.

In [13]:
mc.validate()

mc.save("patra_modelcard.json")
print("Model Card validation successful and file saved.")

Model Card validation successful and file saved.


---
# Conclusion

Congratulations! You have successfully:

1. Trained a neural network on the UCI Adult Dataset using TensorFlow.  
2. Created a **Patra Model Card** capturing essential metadata.  
3. Automatically analyzed bias and generated basic explainability metrics.  
4. Validated and saved the Model Card in JSON format.

This process is a foundation for more advanced use-cases, such as:
- Uploading the Model Card to the **Patra Knowledge Base** for search and provenance tracking.
- Performing deeper fairness analysis (e.g., multiple protected attributes).
- Integrating advanced interpretability approaches.

By consistently generating and maintaining Model Cards, you’ll be on your way to creating **more transparent** and **accountable** AI solutions.


---

# Uploading the TensorFlow model to Hugging Face Hub

In [22]:
!pip install huggingface_hub

from huggingface_hub import create_repo, create_commit, CommitOperationAdd, notebook_login, upload_file, hf_hub_download

notebook_login()

In [23]:
# Save the model in HDF5 format
model_save_name = "tensorflow_model.h5"
model.save(model_save_name)

In [24]:
username = "nkarthikeyan"
repo_name = "IubNet"
repo_id = f"{username}/{repo_name}"

# Create the repository on Hugging Face Hub
create_repo(repo_id=repo_id, private=False, exist_ok=True)

RepoUrl('https://huggingface.co/nkarthikeyan/IubNet', endpoint='https://huggingface.co', repo_type='model', repo_id='nkarthikeyan/IubNet')

In [25]:
# Upload the HDF5 model file to the repository
upload_file(
    path_or_fileobj=model_save_name,       # Correct parameter name
    path_in_repo=model_save_name,          # Destination path in the repo
    repo_id=repo_id,                        # Repository ID
    commit_message="Add Keras model via upload_file"
)

CommitInfo(commit_url='https://huggingface.co/nkarthikeyan/IubNet/commit/9c38d9cd40b57b9be824af67db0eb64752540b0c', commit_message='Add Keras model via upload_file', commit_description='', oid='9c38d9cd40b57b9be824af67db0eb64752540b0c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nkarthikeyan/IubNet', endpoint='https://huggingface.co', repo_type='model', repo_id='nkarthikeyan/IubNet'), pr_revision=None, pr_num=None)

# Loading the TensorFlow model from Hugging Face Hub

In [26]:
# Download the model file from the Hub
model_file = hf_hub_download(
    repo_id="nkarthikeyan/IubNet",
    filename="tensorflow_model.h5"
)

# Load the model
loaded_model = tf.keras.models.load_model(model_file)
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,107 (90.27 KB)

 Trainable params: 23,105 (90.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

---

# Uploading the PyTorch model to Hugging Face Hub

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class SimpleFFNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleFFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)  # Shape: (num_samples, 1)
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# Define input size
input_size = X_train.shape[1]

# Initialize the model
model = SimpleFFNN(input_size=input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define input size
input_size = X_train.shape[1]

# Initialize the model
model = SimpleFFNN(input_size=input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
patience = 3  # Early stopping patience

best_loss = float('inf')
counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Early Stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        # Save the best model
        torch.save(model.state_dict(), "pytorch_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

model.load_state_dict(torch.load("pytorch_model.pth"))
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        predicted = (outputs >= 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")



Epoch [1/100], Loss: 75.8730
Epoch [2/100], Loss: 75.9291
Epoch [3/100], Loss: 75.9291
Epoch [4/100], Loss: 75.9291
Early stopping triggered.
Test Accuracy: 0.2412


<ipython-input-27-fdcc0fcbd1a3>:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("pytorch_model.pth"))


In [29]:
model_save_name = "pytorch_model.h5"
torch.save(model.state_dict(), model_save_name)

In [30]:
# Upload the HDF5 model file to the repository
upload_file(
    path_or_fileobj=model_save_name,       # Correct parameter name
    path_in_repo=model_save_name,          # Destination path in the repo
    repo_id=repo_id,                        # Repository ID
    commit_message="Add Keras model via upload_file"
)

pytorch_model.h5:   0%|          | 0.00/95.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nkarthikeyan/IubNet/commit/88cf89d8ea9e8d79217e1da8300f967bbb3860e3', commit_message='Add Keras model via upload_file', commit_description='', oid='88cf89d8ea9e8d79217e1da8300f967bbb3860e3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nkarthikeyan/IubNet', endpoint='https://huggingface.co', repo_type='model', repo_id='nkarthikeyan/IubNet'), pr_revision=None, pr_num=None)

---

# Downloading the PyTorch model to Hugging Face Hub

In [31]:
class SimpleFFNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleFFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out


In [32]:
# Download the model file from the Hub
downloaded_model_path = hf_hub_download(
    repo_id=repo_id,
    filename="pytorch_model.h5"
)

print(f"Model downloaded to: {downloaded_model_path}")

pytorch_model.h5:   0%|          | 0.00/95.7k [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--nkarthikeyan--IubNet/snapshots/88cf89d8ea9e8d79217e1da8300f967bbb3860e3/pytorch_model.h5


In [33]:
# Initialize the model architecture
loaded_model = SimpleFFNN(input_size=input_size)

# Load the state dictionary
loaded_model.load_state_dict(torch.load(downloaded_model_path))

# Set the model to evaluation mode
loaded_model.eval()

# Verify the model architecture
print(loaded_model)

SimpleFFNN(
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


<ipython-input-33-1322f3c77af4>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(downloaded_model_path))


In [34]:
# Create a sample input tensor
sample_input = torch.randn(1, input_size)

# Perform a forward pass
with torch.no_grad():
    output = loaded_model(sample_input)
    print(f"Model Output: {output.item():.4f}")


Model Output: 0.5087
